In [1]:
import pyautogui as gui
gui.PAUSE = 1
import numpy as np
gui.FAILSAFE = True
import random
import time
import mouse as ms
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from collections import namedtuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch
from torch.autograd import variable
import torchvision.models.resnet as resnet

In [2]:
input_size = 7
output_size = 2

In [3]:
game = gui.getWindow('BlueStacks')
pos = game.get_position()

In [4]:
pos

(46, 23, 380, 686)

In [5]:
ln = pos[2] - pos[0]
ht = pos[3] - pos[1]
replay_x = pos[0] + int(ln*0.4940)
replay_y = pos[1] + int(ht*0.7194)
play_x = pos[0] + int(0.5*ln)
play_y = pos[1] + int(0.8733*ht)
red_check_x = int(np.round(pos[0] + ln*0.757485))
red_check_y = int(np.round(pos[1] + ht*0.54901))
red_sq_ln = 0.12574*ln
red_sq_ht = 0.063348*ln

In [18]:
def replay():
    replay_state = cv2.resize(cv2.cvtColor(np.array(gui.grab(region=(pos[0],pos[1],ln,ht))), cv2.COLOR_BGR2GRAY),(12,22))
    ms.move(replay_x,replay_y)
    ms.click()
    time.sleep(0.7)
    ms.click()
    return replay_state

def press():
    ms.move(play_x,play_y)
    ms.press()

def release():
    ms.move(play_x,play_y)
    ms.release()

def action(i):
    if i == 1:
        press()
    if i == 0:
        release()
        
def isdone():
    red_sq = gui.grab(region=(red_check_x,red_check_y,red_sq_ln,red_sq_ht))
    if (np.array(red_sq)[0][12][0] > 100) & (np.array(red_sq)[0][12][1] < 10):
        return True
    else:
        return False
    
def reset():
    ms.release()
    ms.move(int(pos[0]+np.round(ln*0.82035)),int(pos[1]+np.round(ht*0.11463)))
    ms.click()
    time.sleep(0.5)
    ms.move(int(pos[0]+np.round(ln*0.365269)),int(pos[1]+np.round(ht*0.71040)))
    ms.click()
    time.sleep(0.2)
    ms.click()

In [19]:
def step(i):
    image_before = cv2.resize(cv2.cvtColor(np.array(gui.grab(region=(pos[0],pos[1],ln,ht))), cv2.COLOR_BGR2GRAY),(12,22))
    if i == 1:
        if ms.is_pressed() == False:
            action(1)
    elif i == 0:
        if ms.is_pressed():
            action(0)
    #image_after = cv2.resize(cv2.cvtColor(np.array(gui.grab(region=(pos[0],pos[1],ln,ht))), cv2.COLOR_BGR2GRAY),(12,22))
    #if isdone() == False:
    #    reward = 1
    #else:
    #    reward = 0
    return image_before,isdone()

In [21]:
t1 = time.time()
i = 1
reward = 0
image_before = cv2.resize(cv2.cvtColor(np.array(gui.grab(region=(pos[0],pos[1],ln,ht))), cv2.COLOR_BGR2GRAY),(12,22))
if i == 1:
    if ms.is_pressed() == False:
        action(1)
elif i == 0:
    if ms.is_pressed():
        action(0)
#image_after = cv2.resize(cv2.cvtColor(np.array(gui.grab(region=(pos[0],pos[1],ln,ht))), cv2.COLOR_BGR2GRAY),(12,22))
#if isdone() == False:
#    reward = 1
#else:
#    reward = 0
#return image_before,reward,isdone(),image_after
t2 = time.time()
print(t2-t1)

0.049777984619140625


In [22]:
class Net(nn.Module):
    def __init__(self,input_channel,output_channel):
        self.input_channel = input_channel
        self.output_channel = output_channel
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(input_channel,32,3,padding=0)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(32,32,3,padding=0)
        self.batch_norm2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()
        
#        self.res1 = resnet.BasicBlock(32,32)
#        self.res2 = resnet.BasicBlock(32,32)
#        self.res3 = resnet.BasicBlock(32,32)
#        self.res4 = resnet.BasicBlock(32,32)
#        self.res5 = resnet.BasicBlock(32,32)
#        self.res6 = resnet.BasicBlock(32,32)
#        self.res7 = resnet.BasicBlock(32,32)
#        self.res8 = resnet.BasicBlock(32,32)
#        self.res9 = resnet.BasicBlock(32,32)
#        self.res10 = resnet.BasicBlock(32,32)
        
        
        self.conv5 = nn.Conv2d(32,16,3)
        self.batch_norm5 = nn.BatchNorm2d(16)
        self.relu5 = nn.ReLU()
        
        self.conv6 = nn.Conv2d(16,1,3)
        self.batch_norm6 = nn.BatchNorm2d(1)
        self.relu6 = nn.ReLU()
        
        self.fc1 = nn.Linear(56,output_channel)
        
        
    def forward(self,x):
        x = self.relu1(self.batch_norm1(self.conv1(x)))
        x = self.relu2(self.batch_norm2(self.conv2(x)))
#        x = self.res1(x)
#        x = self.res2(x)
#        x = self.res3(x)
#        x = self.res4(x)
#        x = self.res5(x)
#        x = self.res6(x)
#        x = self.res7(x)
#        x = self.res8(x)
#        x = self.res9(x)
#        x = self.res10(x)
        x = self.relu5(self.batch_norm5(self.conv5(x)))
        x = self.relu6(self.batch_norm6(self.conv6(x)))
        x = x.view(-1,56)
        x = self.fc1(x)
        return x

In [23]:
net = Net(input_size,output_size)

In [24]:
net = net.to('cuda:0')

In [25]:
print(net.conv1.weight.type())

torch.cuda.FloatTensor


In [16]:
t1 = time.time()
for i in range(60):
    
    state,is_done = step(random.choice([1,0]))
t2 = time.time()
print(t2-t1)

4.613802671432495


In [28]:

t1 = time.time()
state = replay()
state = state/255
state_batch = [np.zeros((22,12))]*6
state_batch.append(state)
photo = []
act = []
sm = nn.Softmax(dim=1)
for i in range(60):
    move_probs = sm(net(torch.cuda.FloatTensor(state_batch).view(-1,7,22,12))).tolist()[0]
    probs=np.array(move_probs)
    move = np.random.choice([0,1],p=probs/sum(probs))
    state,is_done = step(move)
    state_batch.pop(0)
    state_batch.append(state)
    photo.append(state)
    act.append(move)
t2 = time.time()

In [17]:
len(photo)

60

In [29]:
t2-t1

5.553325414657593

In [30]:
net.conv1.weight.type()

'torch.cuda.FloatTensor'

In [31]:
torch.cuda.empty_cache()

In [ ]:
#net = Net(input_size,output_size)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
epsilon = 0.1
while True:
    sm = nn.Softmax(dim=1)
    state = replay()
    state = state/255
    state_batch = [np.zeros((22,12))]*6
    state_batch.append(state)
#    reward = 0
#    episode_reward = 0
    stuck_list = []
    training_obs = []
    training_acts = []
    while True:
        move_probs = sm(net(torch.cuda.FloatTensor(state_batch).view(-1,7,22,12))).tolist()[0]
        probs=np.array(move_probs)
        move = np.random.choice([0,1],p=probs/sum(probs))
#        move = np.random.choice([move,np.abs(1-move)],p=[1-epsilon,epsilon])
        state,is_done = step(move)
        state_batch.pop(0)
        state_batch.append(state)
#        episode_reward += reward
        training_obs.append(state)
        training_acts.append(move)
        stuck_image = gui.grab(region=(pos[0],pos[1],int(ln),int(ht/2)))
        stuck_list.append(stuck_image)
        if isdone():
            ms.release()
#            training_obs = training_obs[0:-2]
            training_acts_opposite = list(1 - np.array(training_acts[-3:]))
            training_acts = training_acts[0:-3]
            training_acts.extend(training_acts_opposite)
            
            if len(training_acts) > 100:
                print('training')
                training_obs_v = torch.cuda.FloatTensor(np.array(training_obs[:98])/255)
                training_acts_v = torch.cuda.FloatTensor(np.array(training_acts[:98]))
                obs_batch = training_obs_v[:7]
                del training_acts_v[:6]
                act_batch = training_acts_v[0]
                for i in range(14):
                    action_scores_v = net(obs_batch.view(-1,7, 110, 60))
                    loss_v = objective(action_scores_v, variable(act_batch).long())
                    loss_v.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                    print('loss '+ str(loss_v.item()))
                    del training_acts_v[0]
                    del training_obs_v[0]
                training_obs = training_obs[98:]
                training_acts = training_acts[98:]
                
#                episode_reward = 0
            replay()
        else:
            if stuck_list.count(stuck_image)>25:
                ms.release()
                print('stucked ' + str(episode_reward))
#                episode_reward = 0
                reset()
                stuck_list = []
                training_obs = []
                training_acts = []
#    if episode_reward > 100:
#        break

In [127]:
act_batch

tensor([0., 1., 1., 1., 1., 1., 1.])

In [125]:
objective(action_scores_v, variable(act_batch).long())

C:\ProgramData\Anaconda3\lib\site-packages\torch\autograd\__init__.py:167: UserWarning: torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead
  warnings.warn("torch.autograd.variable(...) is deprecated, use torch.tensor(...) instead")


ValueError: Expected input batch_size (1) to match target batch_size (7).

In [121]:
len(training_acts)

437

In [11]:
training_acts_opposite

[0, 0]